# BUILD SPECTRAL CONTRAST

In [1]:
import numpy as np
import librosa
import pandas as pd
from sklearn import preprocessing
from tqdm.notebook import trange, tqdm

### GLOBAL VARIABLES

In [2]:
INPUT_PATH = 'data/processed/submission.npy'
OUTPUT_PATH = 'data/features/submission_spectral_contrast.npy'
SR = 22050

### FUNCTIONS

In [3]:
def wave2spectral(wave, sr, n_fft, hop_length, S, fmin, max_pad_len, normalize, padding_mode):

    #wave, sr = librosa.load(wav_file, mono=True)
    
    if 0 < len(wave): # workaround: 0 length causes error
        wave, _ = librosa.effects.trim(wave) # trim, top_db=default(60)

    if normalize:
        wave = librosa.util.normalize(wave) # normalizing data before mfcc

    # making melspect from signal
    S = librosa.feature.spectral_contrast(y=wave, sr=sr, n_fft=n_fft, hop_length=hop_length, S=S, fmin=fmin)


    if max_pad_len:
        if S.shape[1] > max_pad_len:
            S = S[:,:max_pad_len]
        else:
            pad_width = max_pad_len - S.shape[1]
            S = np.pad(S, pad_width=((0, 0), (0, pad_width)), mode=padding_mode)
    
    #S = librosa.power_to_db(S)
    S = S.astype(np.float32)
   
    return S

In [4]:
def gen_spectral(input_path, output_path, sr, n_fft=2048, hop_length=512, S=None, max_pad_len=None, fmin=200, normalize=True, padding_mode='constant'):
    
    # Load wave files
    wavfiles = np.load(input_path)
    
    # Init Mel vectors
    X_spectral = []
    
    for idx in trange(wavfiles.shape[0]):
        X_spectral.append(wave2spectral(wavfiles[idx, :], sr, n_fft=n_fft, hop_length=hop_length, S=S, 
                                        fmin=fmin, max_pad_len=max_pad_len, 
                             normalize=normalize, padding_mode=padding_mode))
        pass
    
    X_spectral = np.array(X_spectral)
    
    # Saving Mels 
    np.save(output_path, X_spectral)
    
    pass

### RUNING

In [5]:
gen_spectral(INPUT_PATH, OUTPUT_PATH, SR, n_fft=2048, hop_length=512, normalize=True)